In [1]:
import json
import numpy as np
import colorspacious

In [2]:
np.set_printoptions(precision=3, linewidth=100)

# Accessibility comparison to existing color cycles

Minimum perceptual distance metrics are calculated for a variety of commonly-used color cycles.

In [3]:
def score_cycle(color):
    """
    Calculate min delta E for each position in cycle, including CVD sims.
    """
    rgb255 = [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in color]
    rgb = [
        (int(i[:2], 16) / 255, int(i[2:4], 16) / 255, int(i[4:], 16) / 255)
        for i in color
    ]
    min_dists = [100]
    min_dists_typ = [100]
    min_dists_deut = [100]
    min_dists_prot = [100]
    min_dists_trit = [100]
    min_dist = min_dist_typ = min_dist_deut = min_dist_prot = min_dist_trit = 100
    for i in range(1, len(color)):
        for severity in range(1, 101):
            deut = colorspacious.cspace_convert(
                rgb[: i + 1],
                {
                    "name": "sRGB1+CVD",
                    "cvd_type": "deuteranomaly",
                    "severity": severity,
                },
                "sRGB1",
            )
            prot = colorspacious.cspace_convert(
                rgb[: i + 1],
                {"name": "sRGB1+CVD", "cvd_type": "protanomaly", "severity": severity},
                "sRGB1",
            )
            trit = colorspacious.cspace_convert(
                rgb[: i + 1],
                {"name": "sRGB1+CVD", "cvd_type": "tritanomaly", "severity": severity},
                "sRGB1",
            )
            for j in range(i):
                min_dist_typ = min(min_dist_typ, colorspacious.deltaE(rgb[i], rgb[j]))
                min_dist_deut = min(
                    min_dist_deut, colorspacious.deltaE(deut[i], deut[j])
                )
                min_dist_prot = min(
                    min_dist_prot, colorspacious.deltaE(prot[i], prot[j])
                )
                min_dist_trit = min(
                    min_dist_trit, colorspacious.deltaE(trit[i], trit[j])
                )
        min_dists_typ.append(min_dist_typ)
        min_dists_deut.append(min_dist_deut)
        min_dists_prot.append(min_dist_prot)
        min_dists_trit.append(min_dist_trit)
    min_dists = np.min(
        [min_dists_typ, min_dists_deut, min_dists_prot, min_dists_trit], axis=0
    )
    return np.array(
        [min_dists, min_dists_typ, min_dists_deut, min_dists_prot, min_dists_trit]
    )

In [4]:
def calc_j_range(color):
    """
    Calculate minimum and maximum J' in cycle.
    """
    rgb255 = [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in color]
    min_j = 100
    max_j = 0
    for i in range(len(color)):
        jab = colorspacious.cspace_convert(rgb255[i], "sRGB255", "CAM02-UCS")
        min_j = min(min_j, jab[0])
        max_j = max(max_j, jab[0])
    return (min_j, max_j)

In [5]:
colors = {}
results = {}
j_ranges = {}

In [6]:
def process_cycle(name, cycle):
    """
    Run and store and print results of calculation for cycle.
    """
    colors[name] = cycle
    results[name] = score_cycle(colors[name])
    j_ranges[name] = calc_j_range(colors[name])
    print("     min delta E:", results[name][0])
    print(" min delta E typ:", results[name][1])
    print("min delta E deut:", results[name][2])
    print("min delta E prot:", results[name][3])
    print("min delta E trit:", results[name][4])
    print("    J min, J max:", np.array(j_ranges[name]))

### This Work

We start with the new cycles developed in the present work.

In [7]:
with open("../aesthetic-models/top-cycles.json") as infile:
     new_cycles = json.load(infile)

In [8]:
process_cycle("This Work 6", new_cycles["6"])

     min delta E: [100.     57.138  21.312  21.312  21.312  20.463]
 min delta E typ: [100.     64.545  34.475  32.056  28.818  23.931]
min delta E deut: [100.     63.876  21.312  21.312  21.312  21.312]
min delta E prot: [100.     61.08   31.828  26.007  26.007  20.463]
min delta E trit: [100.     57.138  27.256  27.256  26.6    23.959]
    J min, J max: [41.3   76.435]


In [9]:
process_cycle("This Work 8", new_cycles["8"])

     min delta E: [100.     66.905  18.186  18.056  18.056  18.056  18.056  18.056]
 min delta E typ: [100.     77.98   20.022  20.022  20.022  20.022  20.022  20.022]
min delta E deut: [100.     77.199  18.657  18.657  18.263  18.263  18.067  18.067]
min delta E prot: [100.     66.905  20.037  19.159  19.159  19.159  18.455  18.455]
min delta E trit: [100.     72.963  18.186  18.056  18.056  18.056  18.056  18.056]
    J min, J max: [40.221 78.495]


In [10]:
process_cycle("This Work 10", new_cycles["10"])

     min delta E: [100.     56.751  33.424  22.26   18.316  16.398  16.283  16.053  16.053  16.053]
 min delta E typ: [100.     64.186  41.396  26.611  26.611  22.824  19.747  19.185  18.703  18.703]
min delta E deut: [100.     62.858  33.424  26.627  18.316  16.398  16.283  16.053  16.053  16.053]
min delta E prot: [100.     59.523  39.711  25.688  24.123  20.297  19.387  16.545  16.545  16.545]
min delta E trit: [100.     56.751  37.373  22.26   22.26   19.778  17.407  16.474  16.474  16.474]
    J min, J max: [41.328 83.704]


### Seaborn

[Seaborn](https://seaborn.pydata.org/)'s default color cycle, "Seaborn Deep," and the "Seaborn Colorblind" cycle are evaluated. These are from v0.11.2.

In [11]:
process_cycle("Seaborn Deep", ["4C72B0", "DD8452", "55A868", "C44E52", "8172B3", "937860", "DA8BC3", "8C8C8C", "CCB974", "64B5CD"])

     min delta E: [100.     42.351   5.322   5.322   2.39    2.39    2.39    2.39    2.39    2.39 ]
 min delta E typ: [100.     51.373  37.208  19.395  14.091  14.091  14.091  13.125  13.125  13.125]
min delta E deut: [100.     49.085   9.764   8.803   6.283   4.175   4.175   4.175   4.175   4.175]
min delta E prot: [100.     42.351   5.322   5.322   2.39    2.39    2.39    2.39    2.39    2.39 ]
min delta E trit: [100.     47.514  16.008  15.449  12.813  12.813   9.997   9.568   9.568   9.568]
    J min, J max: [49.443 78.224]


In [12]:
process_cycle("Seaborn Colorblind", ["0173B2", "DE8F05", "029E73", "D55E00", "CC78BC", "CA9161", "FBAFE4", "949494", "ECE133", "56B4E9"])

     min delta E: [100.     55.827  12.819  10.62    9.94    5.836   5.836   5.836   5.836   5.836]
 min delta E typ: [100.     62.438  34.226  16.286  16.286  12.988  12.988  12.988  12.988  12.988]
min delta E deut: [100.     60.722  26.411  10.62   10.62   10.62   10.62    8.458   8.458   8.458]
min delta E prot: [100.     56.441  21.173  14.01   10.642   5.836   5.836   5.836   5.836   5.836]
min delta E trit: [100.     55.827  12.819  12.819   9.94    6.088   6.088   6.088   6.088   6.088]
    J min, J max: [47.103 89.608]


### Tableau Software

The "Category 10," "Tableau Color Blind," and "Tableau 10" cycles are evaluated. The "Category 10" cycle originated at [Tableau Software](https://www.tableau.com/) but is presently the default in a number of other plotting codes, including [Matplotlib](https://matplotlib.org/). The "Tableau Color Blind" cycle is also available in Tableau. The "Tableau 10" cycle replaced the "Category 10" cycle as the default in Tableau in [version 10](https://www.tableau.com/about/blog/2016/7/colors-upgrade-tableau-10-56782).

In [13]:
process_cycle("Category 10", ["1f77b4", "ff7f0e", "2ca02c", "d62728", "9467bd", "8c564b", "e377c2", "7f7f7f", "bcbd22", "17becf"])

     min delta E: [100.     54.081   3.433   3.433   1.961   1.961   1.961   1.961   1.961   1.961]
 min delta E typ: [100.     65.69   46.779  26.147  26.147  23.673  22.932  20.213  20.213  20.213]
min delta E deut: [100.     61.184  16.687   5.157   5.157   5.157   5.157   5.157   4.462   4.293]
min delta E prot: [100.     54.081   3.433   3.433   1.961   1.961   1.961   1.961   1.961   1.961]
min delta E trit: [100.     59.927  13.395  13.395  13.395  13.395  12.663  11.126  11.126  11.126]
    J min, J max: [45.849 76.782]


In [14]:
process_cycle("Tableau Color Blind", ["1170aa", "fc7d0b", "a3acb9", "57606c", "5fa2ce", "c85200", "7b848f", "a3cce9", "ffbc79", "c8d0d9"])

     min delta E: [100.     54.004  27.896  16.817  12.14   12.14   12.14   11.621  11.621   8.326]
 min delta E typ: [100.     66.118  32.224  18.404  16.684  16.684  14.917  13.15   13.15   11.245]
min delta E deut: [100.     61.582  32.097  17.382  15.28   15.28   14.918  11.621  11.621  10.128]
min delta E prot: [100.     54.004  27.896  16.817  12.14   12.14   12.14   12.14   12.14    8.326]
min delta E trit: [100.     60.442  31.571  17.622  15.804  15.804  14.902  12.723  12.723  10.467]
    J min, J max: [42.345 85.08 ]


In [15]:
process_cycle("Tableau 10", ["4e79a7", "f28e2b", "e15759", "76b7b2", "59a14f", "edc948", "b07aa1", "ff9da7", "9c755f", "bab0ac"])

     min delta E: [100.     48.514  13.721  13.721   0.792   0.792   0.792   0.792   0.792   0.792]
 min delta E typ: [100.     56.463  23.424  23.424  23.424  19.255  19.255  19.255  19.255  19.255]
min delta E deut: [100.     53.998  16.515  16.515   0.792   0.792   0.792   0.792   0.792   0.792]
min delta E prot: [100.     48.514  22.857  22.857   7.631   7.631   7.394   4.038   2.164   1.394]
min delta E trit: [100.     51.166  13.721  13.721  11.261  11.261  11.261   8.232   6.693   6.693]
    J min, J max: [51.215 84.669]


### Microsoft Excel

This cycle is the default in Microsoft Excel 2019 (build 2110), where it is also referred to as "Colorful Palette 1."

In [16]:
process_cycle("Microsoft Excel", ["417ebf", "e38248", "a5a6a7", "faba45", "5da3d1", "7ca353", "224873", "964b29", "616263", "95702e"])

     min delta E: [100.     44.408  23.585  14.945  12.812   4.052   4.052   4.052   4.052   4.052]
 min delta E typ: [100.     54.708  29.071  19.426  13.959  13.959  13.959  13.959  13.959  13.959]
min delta E deut: [100.     51.158  24.22   14.945  13.664   6.983   6.983   6.983   6.983   6.983]
min delta E prot: [100.     44.408  23.585  18.062  13.604   4.052   4.052   4.052   4.052   4.052]
min delta E trit: [100.     50.362  26.255  16.024  12.812  11.234  11.234  11.234  11.234  11.234]
    J min, J max: [31.066 82.931]


### MATLAB

This cycle is the default in [MATLAB R2021b](https://www.mathworks.com/help/matlab/ref/colororder.html).

In [17]:
process_cycle("MATLAB", ["0072BD", "D95319", "EDB120", "7E2F8E", "77AC30", "4DBEEE", "A2142F"])

     min delta E: [100.     49.707  20.698   9.978   7.694   7.694   7.694]
 min delta E typ: [100.     63.463  30.74   30.74   25.609  25.609  23.301]
min delta E deut: [100.     56.387  20.698   9.978   7.694   7.694   7.694]
min delta E prot: [100.     49.707  27.042  16.776   7.918   7.918   7.918]
min delta E trit: [100.     58.427  26.736  26.736  23.594  20.391  18.854]
    J min, J max: [38.352 79.303]


### LibreOffice Calc

This cycle is the default in [LibreOffice Calc v7.2](https://github.com/LibreOffice/core/blob/libreoffice-7.2.3.2/extras/source/palettes/chart-palettes.soc).

In [18]:
process_cycle("LibreOffice Calc", ["004586", "ff420e", "ffd320", "579d1c", "7e0021", "83caff", "314004", "aecf00", "4b1f6f", "ff950e"])

     min delta E: [100.     51.978  23.789   8.047   8.047   8.047   5.092   4.692   4.692   3.334]
 min delta E typ: [100.     72.956  42.74   35.588  35.588  35.588  35.588  16.749  16.749  16.749]
min delta E deut: [100.     66.167  23.789   8.047   8.047   8.047   5.092   5.092   5.092   3.334]
min delta E prot: [100.     51.978  34.878  12.628  12.628  12.628  12.628   4.692   4.692   4.692]
min delta E trit: [100.     68.378  37.781  31.861  31.861  23.532  22.94   15.348  15.348  15.348]
    J min, J max: [25.082 88.394]


### Google Sheets

This cycle was the default in Google Sheets as of 2021-11-21.

In [19]:
process_cycle("Google Sheets", ["4285f4", "ea4335", "fbbc04", "34a853", "ff6d01", "46bdc6", "7baaf7", "f07b72", "fcd04f", "71c287"])

     min delta E: [100.     50.405  24.186   7.841   7.841   7.841   6.609   6.609   6.038   5.39 ]
 min delta E typ: [100.     64.771  39.683  37.962  15.55   15.55   14.508  14.508   6.792   6.792]
min delta E deut: [100.     56.784  24.186   7.841   7.841   7.841   7.841   7.841   6.216   5.39 ]
min delta E prot: [100.     50.405  34.943  18.271  11.801  11.801  11.801  11.46    6.785   6.785]
min delta E trit: [100.     60.846  31.285  12.276  10.09   10.09    6.609   6.609   6.038   6.038]
    J min, J max: [57.735 87.74 ]


### R

This cycle is the default in R v4.2.1 and was copied from `R-4.2.1/src/library/grDevices/R/colorstuff.R`. It became the default in [R v4](https://developer.r-project.org/Blog/public/2019/11/21/a-new-palette-for-r/). Note that the popular R plotting library [ggplot2](https://ggplot2.tidyverse.org/) uses different, and much less accessible, colors.

In [20]:
process_cycle("R", ["000000", "df536b", "61d04f", "2297e6", "28e2e5", "cd0bbc", "f5c710", "9e9e9e"])

     min delta E: [100.     50.444  17.736  17.736  12.712   8.291   8.27    8.27 ]
 min delta E typ: [100.     68.333  59.082  53.046  29.993  24.75   24.75   24.75 ]
min delta E deut: [100.     62.432  17.736  17.736  17.736   8.291   8.291   8.291]
min delta E prot: [100.     50.444  36.539  33.142  28.086  20.827   8.27    8.27 ]
min delta E trit: [100.     66.655  51.79   19.077  12.712  10.003  10.003  10.003]
    J min, J max: [5.54e-22 8.48e+01]


### Okabe and Ito

This is the cycle developed by [Okabe and Ito (2002)](https://jfly.uni-koeln.de/color/) (also see [archived original page](https://web.archive.org/web/20030821055411/http://jfly.iam.u-tokyo.ac.jp/color/text.html)). It was also published without attribution in [Wong (2011)](https://doi.org/10.1038/nmeth.1618).

In [21]:
process_cycle("Okabe and Ito", ["000000", "E69F00", "56B4E9", "009E73", "F0E442", "0072B2", "D55E00", "CC79A7"])

     min delta E: [100.     77.172  49.341  13.768  13.768  13.109  13.109  10.971]
 min delta E typ: [100.     80.788  56.831  31.527  20.755  20.755  20.755  20.755]
min delta E deut: [100.     80.747  54.562  29.463  15.853  15.853  15.087  14.766]
min delta E prot: [100.     77.172  54.101  23.738  18.586  18.586  18.586  13.955]
min delta E trit: [100.     78.454  49.341  13.768  13.768  13.109  13.109  10.971]
    J min, J max: [5.54e-22 9.07e+01]


### ColorBrewer

This is the "Set 1" cycle from [ColorBrewer](https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=9).

In [22]:
process_cycle("ColorBrewer Set 1", ["e41a1c", "377eb8", "4daf4a", "984ea3", "ff7f00", "ffff33", "a65628", "f781bf", "999999"])

     min delta E: [100.     44.676  11.358   4.656   4.656   4.656   3.456   3.456   3.456]
 min delta E typ: [100.     64.953  45.946  33.144  25.305  25.305  20.6    20.6    20.6  ]
min delta E deut: [100.     51.069  11.358   4.656   4.656   4.656   4.656   4.656   4.656]
min delta E prot: [100.     44.676  27.266  10.24    9.223   9.223   3.456   3.456   3.456]
min delta E trit: [100.     60.033  15.588  15.588  15.588  15.588  13.117   9.833   9.833]
    J min, J max: [49.108 97.522]


### Mathematica

This is the default cycle in Mathematica as of v12.

In [23]:
process_cycle("Mathematica", ["5e81b5", "e19c24", "8fb032", "eb6235", "8778b3", "c56e1a", "5d9ec7", "ffbf00", "a5609d", "929600"])

     min delta E: [100.     46.943   2.128   2.128   1.145   1.145   1.145   1.145   1.145   1.145]
 min delta E typ: [100.     54.2    21.861  21.757  13.211  12.698  11.056  11.056  11.056   9.026]
min delta E deut: [100.     53.611   5.725   4.27    2.521   2.521   2.521   2.521   2.521   2.331]
min delta E prot: [100.     51.286   2.128   2.128   1.145   1.145   1.145   1.145   1.145   1.145]
min delta E trit: [100.     46.943  19.468  16.869  11.783  10.443   9.76    9.76    9.76    8.742]
    J min, J max: [54.399 84.228]


### Scientific colour maps

This is from Fabio Crameri's [set of colormaps](https://doi.org/10.5281/zenodo.5501399).

In [24]:
process_cycle("batlowS", ["011959", "faccfa", "828231", "226061", "f19d6b", "fdb4b4", "114360", "4d734d", "c09036", "175262"])

     min delta E: [100.     72.43   35.563  23.773  16.838   9.046   9.046   9.046   8.431   5.141]
 min delta E typ: [100.     78.449  50.174  31.393  29.386  14.029  14.029  14.029  14.029   7.547]
min delta E deut: [100.     78.028  46.769  29.318  21.362  13.887  13.887  13.887  10.951   6.577]
min delta E prot: [100.     72.43   49.044  29.922  16.838  14.055  14.055  13.124   8.431   6.882]
min delta E trit: [100.     78.011  35.563  23.773  20.663   9.046   9.046   9.046   9.046   5.141]
    J min, J max: [14.351 89.627]


### Plots.jl

This is the default cycle from the [Plots.jl](https://docs.juliaplots.org/latest/generated/colorschemes/) Julia library, v1.24.3.

In [25]:
process_cycle("Plots.jl", ["009AFA", "E36F47", "3EA44E", "C371D2", "AC8E18", "00AAAE", "ED5E93", "C68225", "00A98D", "8E971E"])

     min delta E: [100.     48.476   7.209   7.209   5.072   5.072   3.774   1.076   1.076   1.076]
 min delta E typ: [100.     60.76   45.328  36.713  24.349  22.745  20.057  10.026  10.026   8.95 ]
min delta E deut: [100.     53.822   7.209   7.209   5.072   5.072   5.072   1.076   1.076   1.076]
min delta E prot: [100.     48.476   8.226   8.226   8.226   8.226   8.226   3.168   3.168   1.285]
min delta E trit: [100.     56.508  13.948  13.948  10.077   6.169   3.774   3.774   2.492   2.492]
    J min, J max: [61.252 65.71 ]


### Colour Schemes

This cycle is from Paul Tol's [SRON/EPS/TN/09-002 technical report](https://personal.sron.nl/~pault/data/colourschemes.pdf), issue 3.2.

In [26]:
process_cycle("Tol Bright", ["4477AA", "EE6677", "228833", "CCBB44", "66CCEE", "AA3377", "BBBBBB"])

     min delta E: [100.     24.978   7.369   7.369   7.369   7.369   7.369]
 min delta E typ: [100.     51.304  41.164  34.299  29.945  23.685  23.38 ]
min delta E deut: [100.     39.121  17.334  17.334  17.334  16.155  16.155]
min delta E prot: [100.     24.978  18.15   18.15   18.15   15.434  15.434]
min delta E trit: [100.     47.914   7.369   7.369   7.369   7.369   7.369]
    J min, J max: [46.409 78.435]


### Plotly Express

This is the default cycle from the Plotly Express, which is part of the plotly.py Python library, v5.4.0.

In [27]:
process_cycle("Plotly", ["636EFA", "EF553B", "00CC96", "AB63FA", "FFA15A", "19D3F3", "FF6692", "B6E880", "FF97FF", "FECB52"])

     min delta E: [100.     52.518  19.255   1.014   1.014   1.014   1.014   1.014   1.014   1.014]
 min delta E typ: [100.     61.913  52.289  18.338  18.338  18.338  17.088  17.088  17.088  15.37 ]
min delta E deut: [100.     60.039  19.255   8.02    8.02    8.02    2.872   2.872   2.872   2.872]
min delta E prot: [100.     52.518  26.57    1.014   1.014   1.014   1.014   1.014   1.014   1.014]
min delta E trit: [100.     58.367  22.634  17.062  17.062   6.986   6.986   6.986   6.986   6.986]
    J min, J max: [54.287 87.616]


## Generate table

The results are processed into a LaTeX table.

In [28]:
names = [
    "This Work 6",
    "This Work 8",
    "This Work 10",
    "ColorBrewer Set 1",
    "batlowS",

    "Microsoft Excel",
    "LibreOffice Calc",
    "Google Sheets",
    "Mathematica",
    "Plots.jl",

    "Plotly",
    "Seaborn Deep",
    "Tableau 10",
    "Category 10",
    "MATLAB",

    "R",
    "Seaborn Colorblind",
    "Tableau Color Blind",
    "Okabe and Ito",
    "Tol Bright",
]

In [29]:
emph_text = r"\textcolor[HTML]{990099}{\textbf{"
threshold = 10
rows = 4
cols = 5
output = r"\begin{tabular}{@{}" + "crr" * cols + "@{}}\n"
for k in range(rows):
    output += r"\toprule" + "\n"
    for i in range(cols):
        output += r"\multicolumn{3}{c}{" + names[k * cols + i] + "} & "
    output = output[:-2] + r"\\" + "\n"
    for i in range(cols):
        output += (
            r"\multicolumn{3}{c}{$J' \in ["
            + f"{j_ranges[names[k * cols + i]][0]:.1f}, {j_ranges[names[k * cols + i]][1]:.1f}"
            + "]$} & "
        )
    output = output[:-2]
    output += (
        r"\\ \cmidrule(r){1-3}\cmidrule(rl){4-6}\cmidrule(rl){7-9}\cmidrule(rl){10-12}\cmidrule(l){13-15}"
        + "\n"
    )
    output += r"& $\min$ & $\min$ & " * (cols - 1) + r"& $\min$ & $\min$ \\" + "\n"
    output += (
        r"& $\Delta E'$ & $\Delta E_\text{cvd}$ & " * (cols - 1)
        + r"& $\Delta E'$ & $\Delta E_\text{cvd}$ \\ \midrule"
        + "\n"
    )
    for i in range(10):
        for j in range(cols):
            if i < len(colors[names[k * cols + j]]):
                output += (
                    r"\textcolor[HTML]{"
                    + colors[names[k * cols + j]][i]
                    + r"}{$\blacksquare$} & "
                )
                if results[names[k * cols + j]][1][i] <= threshold:
                    output += emph_text
                output += f"{results[names[k * cols + j]][1][i]:.1f}"
                if results[names[k * cols + j]][1][i] <= threshold:
                    output += "}}"
                output += " & "
                if results[names[k * cols + j]][0][i] <= threshold:
                    output += emph_text
                output += f"{results[names[k * cols + j]][0][i]:.1f}"
                if results[names[k * cols + j]][0][i] <= threshold:
                    output += "}}"
                output += " & "
            else:
                output += "& & & "
        output = output[:-2] + "\\\\\n"
output += r"""\bottomrule
\end{tabular}"""
print(output)

\begin{tabular}{@{}crrcrrcrrcrrcrr@{}}
\toprule
\multicolumn{3}{c}{This Work 6} & \multicolumn{3}{c}{This Work 8} & \multicolumn{3}{c}{This Work 10} & \multicolumn{3}{c}{ColorBrewer Set 1} & \multicolumn{3}{c}{batlowS} \\
\multicolumn{3}{c}{$J' \in [41.3, 76.4]$} & \multicolumn{3}{c}{$J' \in [40.2, 78.5]$} & \multicolumn{3}{c}{$J' \in [41.3, 83.7]$} & \multicolumn{3}{c}{$J' \in [49.1, 97.5]$} & \multicolumn{3}{c}{$J' \in [14.4, 89.6]$} \\ \cmidrule(r){1-3}\cmidrule(rl){4-6}\cmidrule(rl){7-9}\cmidrule(rl){10-12}\cmidrule(l){13-15}
& $\min$ & $\min$ & & $\min$ & $\min$ & & $\min$ & $\min$ & & $\min$ & $\min$ & & $\min$ & $\min$ \\
& $\Delta E'$ & $\Delta E_\text{cvd}$ & & $\Delta E'$ & $\Delta E_\text{cvd}$ & & $\Delta E'$ & $\Delta E_\text{cvd}$ & & $\Delta E'$ & $\Delta E_\text{cvd}$ & & $\Delta E'$ & $\Delta E_\text{cvd}$ \\ \midrule
\textcolor[HTML]{5790fc}{$\blacksquare$} & 100.0 & 100.0 & \textcolor[HTML]{1845fb}{$\blacksquare$} & 100.0 & 100.0 & \textcolor[HTML]{3f90da}{$\blacksqu

In [30]:
with open("cycle-comparison.json", "w") as outfile:
    json.dump({"colors": colors, "results": {r: results[r].tolist() for r in results}, "j_ranges": j_ranges}, outfile, indent=2)